In [3]:
import pandas as pd

# Load accredited trainers
trainers_df = pd.read_excel("./AccreditedTrainers.xlsx")

# Load accredited courses
courses_df = pd.read_excel("./AccreditedCourses.xlsx")

# Preview column names
print("=== Trainers ===")
print(trainers_df.columns)
print("\n=== Courses ===")
print(courses_df.columns)

=== Trainers ===
Index(['S.No.', 'Reg. No.', 'Name', 'County', 'Gender', 'Type',
       'Highest Qualification', 'Approved Level', 'Cluster',
       'Area of specialization', 'Expiry Date'],
      dtype='object')

=== Courses ===
Index(['Sno', 'CourseName', 'Level', 'ExamBody', 'Registration number',
       'Institution'],
      dtype='object')


In [4]:
# === STEP 2A: Assign KNQA Level for Trainers ===

def map_highest_qualification_to_knqa(qualification):
    qualification = str(qualification).lower().strip()
    if "phd" in qualification or "doctor" in qualification:
        return 10
    elif "master" in qualification:
        return 9
    elif "post graduate diploma" in qualification:
        return 8
    elif "Bachelor" in qualification:
        return 7
    elif "higher national diploma" in qualification or "higher diploma" in qualification:
        return 6
    elif "diploma" in qualification:
        return 6
    elif "craft" in qualification or "professional certificate" in qualification:
        return 5
    elif "artisan" in qualification or "vocational certificate iii" in qualification:
        return 4
    elif "certificate ii" in qualification:
        return 3
    elif "certificate i" in qualification or "secondary" in qualification:
        return 2
    elif "primary" in qualification:
        return 1
    else:
        return None

trainers_df['KNQA_Level_trainer'] = trainers_df['Highest Qualification'].apply(map_highest_qualification_to_knqa)

# === STEP 2B: Assign KNQA Level for Courses ===

def map_course_level_to_knqa(level):
    level = str(level).lower().strip()
    if "short course" in level:
        return 1
    elif level == "level":
        return None
    elif "artisan" in level:
        return 4
    elif "craft" in level or "national certificate" in level:
        return 5
    elif "higher diploma" in level or "higher national diploma" in level:
        return 6
    elif "diploma" in level:
        return 6
    elif "level 6" in level:
        return 6
    elif "level 5" in level:
        return 5
    elif "level 4" in level:
        return 4
    elif "level 3" in level:
        return 3
    elif "level 1" in level or "level 2" in level or "1and2" in level:
        return 2
    elif "nvcet" in level:
        return 4
    elif "professional" in level:
        return 5
    else:
        return None

courses_df['KNQA_Level_course'] = courses_df['Level'].apply(map_course_level_to_knqa)

In [5]:
# Merge on KNQA level
merged_df = pd.merge(
    courses_df,
    trainers_df,
    left_on='KNQA_Level_course',
    right_on='KNQA_Level_trainer',
    how='inner',
    suffixes=('_course', '_trainer')
)

# Confirm the shape
print("Merged shape:", merged_df.shape)

# Preview a few key columns
print(merged_df[['CourseName', 'Level', 'KNQA_Level_course', 'Name', 'Highest Qualification', 'KNQA_Level_trainer']].head())

Merged shape: (9469570, 19)
                             CourseName  Level  KNQA_Level_course  \
0  Information Communication Technology  Craft                5.0   
1  Information Communication Technology  Craft                5.0   
2  Information Communication Technology  Craft                5.0   
3  Information Communication Technology  Craft                5.0   
4  Information Communication Technology  Craft                5.0   

             Name     Highest Qualification  KNQA_Level_trainer  
0  Kennedy Barasa                     Craft                 5.0  
1   Hellen Mutiso  Professional Certificate                 5.0  
2  Jason Githinji  Professional Certificate                 5.0  
3   Jocelyne Orie  Professional Certificate                 5.0  
4     Jane Githua  Professional Certificate                 5.0  


In [6]:
import numpy as np

In [7]:
# Drop duplicate courses (retain only course-level info)
courses_clean = courses_df.drop_duplicates(subset=[
    'CourseName', 'Level', 'ExamBody', 'Institution'
]).copy()

# Assign KNQA_Level again (if not already)
courses_clean['KNQA_Level_course'] = courses_clean['Level'].apply(map_course_level_to_knqa)

# Extract institution type
def extract_type(name):
    name = str(name).lower()
    if 'polytechnic' in name:
        return 'National Polytechnic'
    elif 'technical' in name or 'institute' in name:
        return 'Technical Institute'
    elif 'university' in name:
        return 'University'
    elif 'college' in name:
        return 'College'
    else:
        return 'Other'

courses_clean['Institution_Type'] = courses_clean['Institution'].apply(extract_type)

# Assign rule-based trainer demand (e.g. higher level = more trainers)
np.random.seed(42)
courses_clean['Trainer_Demand'] = courses_clean['KNQA_Level_course'].apply(
    lambda level: int(level) + np.random.choice([0, 1]) if not pd.isna(level) else 1
)

# Rename for modeling
courses_clean = courses_clean.rename(columns={'CourseName': 'Course_Name'})

# Final ML dataset
ml_df = courses_clean[[
    'KNQA_Level_course', 'Course_Name', 'ExamBody', 'Institution_Type', 'Trainer_Demand'
]].dropna()

In [8]:
from sklearn.model_selection import train_test_split

# Define features and target
X = ml_df[['KNQA_Level_course', 'Course_Name', 'ExamBody', 'Institution_Type']]
y = ml_df['Trainer_Demand']

# Split the dataset (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Specify which features are categorical
categorical_features = ['KNQA_Level_course', 'Course_Name', 'ExamBody', 'Institution_Type']

# Define the preprocessing pipeline
preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

# Build the pipeline
linear_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Fit the model
linear_pipe.fit(X_train, y_train)

# Predict
y_pred_linear = linear_pipe.predict(X_test)

In [11]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Evaluate Linear Regression model
mae_linear = mean_absolute_error(y_test, y_pred_linear)
mse_linear = mean_squared_error(y_test, y_pred_linear)
r2_linear = r2_score(y_test, y_pred_linear)

# Display results
print("Linear Regression Results:")
print("MAE:", round(mae_linear, 3))
print("MSE:", round(mse_linear, 3))
print("R² Score:", round(r2_linear, 3))

Linear Regression Results:
MAE: 0.499
MSE: 0.258
R² Score: 0.728


In [12]:
from sklearn.ensemble import RandomForestRegressor

# Build pipeline with Random Forest
rf_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Train
rf_pipe.fit(X_train, y_train)

# Predict
y_pred_rf = rf_pipe.predict(X_test)

# Evaluate
mae_rf = mean_absolute_error(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

# Show results
print("Random Forest Results:")
print("MAE:", round(mae_rf, 3))
print("MSE:", round(mse_rf, 3))
print("R² Score:", round(r2_rf, 3))

Random Forest Results:
MAE: 0.497
MSE: 0.267
R² Score: 0.719


In [13]:
from sklearn.ensemble import GradientBoostingRegressor

# Build pipeline with Gradient Boosting
gb_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor(random_state=42))
])

# Train
gb_pipe.fit(X_train, y_train)

# Predict
y_pred_gb = gb_pipe.predict(X_test)

# Evaluate
mae_gb = mean_absolute_error(y_test, y_pred_gb)
mse_gb = mean_squared_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)

# Show results
print("Gradient Boosting Results:")
print("MAE:", round(mae_gb, 3))
print("MSE:", round(mse_gb, 3))
print("R² Score:", round(r2_gb, 3))

Gradient Boosting Results:
MAE: 0.498
MSE: 0.25
R² Score: 0.737


In [14]:
import pandas as pd

# Compile results
results_df = pd.DataFrame([
    {
        "Model": "Linear Regression",
        "MAE": round(mae_linear, 3),
        "MSE": round(mse_linear, 3),
        "R² Score": round(r2_linear, 3)
    },
    {
        "Model": "Random Forest",
        "MAE": round(mae_rf, 3),
        "MSE": round(mse_rf, 3),
        "R² Score": round(r2_rf, 3)
    },
    {
        "Model": "Gradient Boosting",
        "MAE": round(mae_gb, 3),
        "MSE": round(mse_gb, 3),
        "R² Score": round(r2_gb, 3)
    }
])

# Display results table
print(results_df)

               Model    MAE    MSE  R² Score
0  Linear Regression  0.499  0.258     0.728
1      Random Forest  0.497  0.267     0.719
2  Gradient Boosting  0.498  0.250     0.737


In [17]:
# New course input
new_input = pd.DataFrame([{
    'KNQA_Level_course': 4,
    'Course_Name': 'Information Communications Technology',
    'ExamBody': 'KASNEB',
    'Institution_Type': 'Technical Institute'
}])

# Predict using Gradient Boosting model
predicted_demand = gb_pipe.predict(new_input)
print("Predicted Trainer Demand:",int(predicted_demand[0]))

Predicted Trainer Demand: 4


In [16]:
### Create a Pickle file using serialization 
import pickle
pickle_out = open("gb_model.pkl","wb")
pickle.dump(gb_pipe, pickle_out)
pickle_out.close()